<a href="https://colab.research.google.com/github/Jaydeep4845/Sales-data-analysis-/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = "/content/drive/My Drive/Career/Book/Codes/PBJ_Daily_Nurse_Staffing_Q2_2024.csv"
import pandas as pd
import numpy as np
import matplotlib as mpl
df = pd.read_csv(file_path)
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-5-b3ab1175f31c>:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,PROVNUM,PROVNAME,CITY,STATE,COUNTY_NAME,COUNTY_FIPS,CY_Qtr,WorkDate,MDScensus,Hrs_RNDON,...,Hrs_LPN_ctr,Hrs_CNA,Hrs_CNA_emp,Hrs_CNA_ctr,Hrs_NAtrn,Hrs_NAtrn_emp,Hrs_NAtrn_ctr,Hrs_MedAide,Hrs_MedAide_emp,Hrs_MedAide_ctr
0,15009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240401,51,10.77,...,0.0,160.08,160.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240402,52,8.43,...,0.0,135.95,135.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240403,53,11.13,...,0.0,150.31,150.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240404,52,12.27,...,0.0,133.01,133.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240405,52,4.95,...,0.0,137.92,137.92,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df = df.drop(columns=['CY_Qtr'])


KeyError: "['CY_Qtr'] not found in axis"

In [ ]:
filtered_data = df[
    (df["Hrs_RN_emp"] + df["Hrs_RN_ctr"] > 0) &
    (df["Hrs_LPN_emp"] + df["Hrs_LPN_ctr"] > 0) &
    (df["Hrs_CNA_emp"] + df["Hrs_CNA_ctr"] > 0)
]
print(filtered_data.shape)

(1206364, 32)


In [ ]:
print(df.shape)

(1325324, 32)


In [ ]:
df1 = filtered_data.drop(columns=[ 'WorkDate'])

In [ ]:
print(type(df1))  # Should be <class 'pandas.DataFrame'>
if df1 is None:
    print("Error: df1 is None! Check previous operations.")


<class 'pandas.core.frame.DataFrame'>


In [ ]:
cols_to_drop = ['Hrs_RNDON', 'Hrs_RNDON_emp', 'Hrs_RNDON_ctr',
                'Hrs_RNadmin', 'Hrs_RNadmin_emp', 'Hrs_RNadmin_ctr']

# Keep only existing columns
df1 = df1.drop([col for col in cols_to_drop if col in df1.columns], axis=1)

print(df1.columns)


Index(['PROVNUM', 'PROVNAME', 'CITY', 'STATE', 'COUNTY_NAME', 'COUNTY_FIPS',
       'MDScensus', 'Hrs_RN', 'Hrs_RN_emp', 'Hrs_RN_ctr', 'Hrs_LPNadmin',
       'Hrs_LPNadmin_emp', 'Hrs_LPNadmin_ctr', 'Hrs_LPN', 'Hrs_LPN_emp',
       'Hrs_LPN_ctr', 'Hrs_CNA', 'Hrs_CNA_emp', 'Hrs_CNA_ctr', 'Hrs_NAtrn',
       'Hrs_NAtrn_emp', 'Hrs_NAtrn_ctr', 'Hrs_MedAide', 'Hrs_MedAide_emp',
       'Hrs_MedAide_ctr'],
      dtype='object')


In [ ]:
df1['ctr_hours %'] = (
    df1[['Hrs_RN_ctr', 'Hrs_LPNadmin_ctr', 'Hrs_LPN_ctr', 'Hrs_CNA_ctr', 'Hrs_NAtrn_ctr', 'Hrs_MedAide_ctr']].sum(axis=1) /
    df1[['Hrs_RN_ctr', 'Hrs_LPNadmin_ctr', 'Hrs_LPN_ctr', 'Hrs_CNA_ctr', 'Hrs_NAtrn_ctr', 'Hrs_MedAide_ctr',
        'Hrs_RN_emp', 'Hrs_LPNadmin_emp', 'Hrs_LPN_emp', 'Hrs_CNA_emp', 'Hrs_NAtrn_emp', 'Hrs_MedAide_emp']].sum(axis=1)
) * 100


In [ ]:
df1['ctr_hours %'].value_counts()[0]


np.int64(709212)

In [ ]:
num_cols = df1.select_dtypes(include=['number']).columns  # Select only numeric columns
df1[num_cols] = df1[num_cols].round().astype(int)  # Convert only numeric columns to int
df1['ctr_hours %']

,ctr_hours %
0,0
1,0
2,0
3,0
4,0
...,...
1325319,0
1325320,0
1325321,0
1325322,0


In [ ]:
df1 = df1[df1['ctr_hours %'] > 50]


In [ ]:
print(df1.shape)

(17156, 26)


In [ ]:
df1['total_ctl_hours'] = df1[['Hrs_RN_ctr', 'Hrs_LPNadmin_ctr', 'Hrs_LPN_ctr', 'Hrs_CNA_ctr', 'Hrs_NAtrn_ctr', 'Hrs_MedAide_ctr']].sum(axis=1)
df1['total_emp_hours'] = df1[['Hrs_RN_emp', 'Hrs_LPNadmin_emp', 'Hrs_LPN_emp', 'Hrs_CNA_emp', 'Hrs_NAtrn_emp', 'Hrs_MedAide_emp']].sum(axis=1)

In [ ]:
df1['total_ctl_hours']

,total_ctl_hours
20660,156
20661,144
20662,132
20667,152
20674,134
...,...
1322467,60
1322487,70
1322488,73
1322492,65


In [ ]:
from google.colab import files

df1.to_csv('PBJ_Daily_Nurse_Staffing_Q2_2024.csv', index=False)  # Save as CSV
files.download('PBJ_Daily_Nurse_Staffing_Q2_2024.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# Assuming df is your dataframe
df_grouped = df1.groupby('PROVNUM', as_index=False).agg(
    total_ctl_hours=('total_ctl_hours', 'sum'),
    total_emp_hours=('total_emp_hours', 'sum'),
    city_count=('CITY', 'count')
)

# Display the first few rows
print(df_grouped.head())


  PROVNUM  total_ctl_hours  total_emp_hours  city_count
0   35285              216              170           3
1   35289              114               99           1
2   45242             1147              710          15
3   45295              681              508           5
4   45359              121              101           1


In [ ]:
df_grouped['% ctr'] = (df_grouped['total_ctl_hours'] * 100) / (df_grouped['total_ctl_hours'] + df_grouped['total_emp_hours'])

In [ ]:
print(df1.shape)

NameError: name 'df1' is not defined

In [ ]:
df_sorted = df_grouped[['PROVNUM', 'total_ctl_hours', 'total_emp_hours', 'city_count']]\
    .sort_values(by=['total_ctl_hours', 'total_emp_hours', 'city_count'], ascending=[False, False, False])
df_sorted.head(15)

,PROVNUM,total_ctl_hours,total_emp_hours,city_count
965,395290,73240,31302,91
403,335791,65860,27618,91
946,395015,58363,43339,86
232,235729.0,56977,20233,91
375,335291,51532,33724,91
25,105910,51363,0,88
318,315174,51208,17667,90
534,395361,48811,26430,91
347,315396.0,44540,0,91
322,315209,42676,16936,91


In [ ]:
df_grouped_by_cities = df1.groupby('CITY',as_index=False).agg(
    total_ctl_hours=('total_ctl_hours','sum'),
    total_emp_hours=('total_emp_hours','sum'),
    provnum = ('PROVNUM','count')
)
print()

NameError: name 'df1' is not defined

In [ ]:

# Assuming df is your dataframe
df_grouped = df1.groupby('PROVNUM', as_index=False).agg(
    total_ctl_hours=('total_ctl_hours', 'sum'),
    total_emp_hours=('total_emp_hours', 'sum'),
    city_count=('CITY', 'count')
)

# Display the first few rows
print(df_grouped.head())


  PROVNUM  total_ctl_hours  total_emp_hours  city_count
0   35285              216              170           3
1   35289              114               99           1
2   45242             1147              710          15
3   45295              681              508           5
4   45359              121              101           1
